In [ ]:
# default_exp core

# snkrfinder

> API details:
    - sets up the data strucures and tools for Snkr Finder

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#from flask import Flask, flash, request, redirect, render_template, send_from_directory
#from sneakapp import app
from numpy import isnan
import numpy as np 
import pandas as pd
import json

#from .model_utils import nearest_neighbor_image_finder

from werkzeug.utils import secure_filename
import os
#app = Flask(__name__)

####  specific imports
#import numpy as np
from tensorflow import keras
from tensorflow.python.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications import imagenet_utils
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from sklearn.neighbors import NearestNeighbors

pwidth = 224
pheight = 224

global graph    
#graph = K.get_default_graph()
graph = tf.compat.v1.get_default_graph()
network_model = MobileNetV2(weights = 'imagenet', include_top = False, input_shape = (pwidth,pheight,3),pooling = 'avg')

#UPLOAD_FOLDER = os.path.join(os.getcwd(),'sneakapp/static/images')
UPLOAD_FOLDER = os.path.join( os.path.dirname( os.path.dirname( os.path.abspath(__file__) )), 'sneakapp/static/images')
app.secret_key = "secret key"
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# database is 
MEDIA_FOLDER = os.path.join( os.path.dirname( os.path.dirname( os.path.abspath(__file__) )), 'database/img')
#print(MEDIA_FOLDER)
MEDIA_FOLDER =  app.config['MEDIA_FOLDER'] = MEDIA_FOLDER



# img_path = os.path.join( app.config['UPLOAD_FOLDER'], '8046698.1858.jpg')
# #Get Features of uploaded image#
# img_data = image.load_img(img_path, target_size=(pwidth, pheight))
# img_vector = image.img_to_array(img_data)
# img_vector = np.expand_dims(img_vector, axis=0)
# img_vector = preprocess_input(img_vector) #Problem here, must be convention of keras to pass by reference?
# img_vector = imagenet_utils.preprocess_input(img_vector)
# # this should prevent having to calculate the network every time... 
# with graph.as_default():
#         global img_features
#         img_features = network_model.predict(img_vector)


    


db_base = 'database' #'../database'
#database_path = os.path.join(db_base,'summary_withFeatures_Feb-02-2020.json')
database_path = os.path.join(db_base,'categories_with_features_Feb-05-2020.json')

database_json = json.load(open(database_path))
database_df = pd.DataFrame(database_json)

ALLOWED_EXTENSIONS = set(['txt', 'pdf', 'png', 'jpg', 'jpeg', 'gif'])

##### pre-compute the nearest neighbors and then just query...
# get 9 nearest neighbors
n_neighs = 9
            
#logic here in case the n exceeds number of items in database
size_filter_frame = database_df.shape[0]
num_neighs = min(n_neighs, size_filter_frame)

#Fit Nearest Neighbor Model
database_features = np.vstack(database_df['image_features'])    
neighs = NearestNeighbors(n_neighbors=num_neighs) 
neighs.fit(database_features)



global filename
~